In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [2]:
# device = torch.device('mps' if torch.backends.mps.is_available() else "cpu") # Use MPS if available, otherwise use CPU

device = torch.device('cpu')

In [3]:
input_size = 784 # 28x28
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001


In [4]:
train_loader = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_loader = torchvision.datasets.MNIST(root='./data', train=False, download=False, transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_loader, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_loader, batch_size=batch_size, shuffle=False)


In [6]:
# examples = iter(train_loader)
# samples, labels = next(examples)
samples, labels = next(iter(train_loader))
print(samples.shape, labels.shape)

torch.Size([100, 1, 28, 28]) torch.Size([100])


In [7]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn. Linear (input_size, hidden_size)
        self.relu = nn. ReLU()
        self.l2 = nn. Linear (hidden_size, num_classes)

    def forward (self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)

In [8]:
# loss and optimizer 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
#traininig loop
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #origin shape: [100, 1, 28, 28]
        #input layer: 784
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

epoch 1/2, step 100/600, loss = 0.5327
epoch 1/2, step 200/600, loss = 0.3393
epoch 1/2, step 300/600, loss = 0.2595
epoch 1/2, step 400/600, loss = 0.2089
epoch 1/2, step 500/600, loss = 0.2494
epoch 1/2, step 600/600, loss = 0.1709
epoch 2/2, step 100/600, loss = 0.1578
epoch 2/2, step 200/600, loss = 0.2002
epoch 2/2, step 300/600, loss = 0.1043
epoch 2/2, step 400/600, loss = 0.2694
epoch 2/2, step 500/600, loss = 0.2387
epoch 2/2, step 600/600, loss = 0.2768


In [10]:
#test 
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)

        #value, index
        _, predictions = torch.max(outputs, 1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'accuracy = {acc}')

accuracy = 95.17
